# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

data = pd.read_csv('result.csv', sep = ',')
data = data.drop(columns = ['Unnamed: 0'])
data

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Ready to Move,Under Construction,New Property,Resale,Furnished,Semi-Furnished,Unfurnished,Flat,Individual House
0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
2,31700000.0,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,2.0,2.0,2.0,1,0,1,0,0,0,1,1,0
4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,0.0,0.0,0.0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,19500000.0,810.0,19.138320,72.810020,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0
6251,22000000.0,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,1.0,1,0,0,1,0,0,1,1,0
6252,20000000.0,750.0,19.144320,72.824111,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,0,1,1,0
6253,11000000.0,700.0,19.047201,72.872225,2.0,2.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0


In [26]:
y = data['price']
X = data.drop(['price'], axis = 1)

In [27]:
y

0       22400000.0
1       35000000.0
2       31700000.0
3       18700000.0
4       13500000.0
           ...    
6250    19500000.0
6251    22000000.0
6252    20000000.0
6253    11000000.0
6254    15000000.0
Name: price, Length: 6255, dtype: float64

In [28]:
X

,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Ready to Move,Under Construction,New Property,Resale,Furnished,Semi-Furnished,Unfurnished,Flat,Individual House
0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
1,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
2,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
3,629.0,19.155756,72.846862,2.0,2.0,2.0,2.0,2.0,1,0,1,0,0,0,1,1,0
4,1090.0,19.177555,72.849887,2.0,2.0,0.0,0.0,0.0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,810.0,19.138320,72.810020,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0
6251,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,1.0,1,0,0,1,0,0,1,1,0
6252,750.0,19.144320,72.824111,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,0,1,1,0
6253,700.0,19.047201,72.872225,2.0,2.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [30]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5004, 17), (5004,), (1251, 17), (1251,))

In [31]:
lr = LinearRegression().fit(X_train, y_train)
lr.predict(X_test)

array([67948329.62665176, -3297204.3012495 , 18531144.9581871 , ...,
       19837801.29864597, 36217514.80827618, 26125927.05124283])

In [32]:
y_pred = lr.predict(X_test)

In [34]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 8878465.629029155
MSE: 219205476474165.78
RMSE: 14805589.36598492
MAPE: 0.7320388797138937
R^2: 0.6257232430102068


In [35]:
len(lr.coef_)
lr.coef_

array([ 2.21740875e+04, -8.14874249e+07, -7.61010708e+07,  1.36802476e+05,
        7.84199154e+06, -3.01829765e+04, -3.01829765e+04, -3.01829765e+04,
       -2.42338853e+06,  2.42338853e+06,  1.59341285e+05, -1.59341285e+05,
        2.33020669e+06, -1.03109093e+06, -1.29911576e+06,  4.45603100e+04,
       -4.45603100e+04])

In [37]:
#L1
ridge = Ridge(alpha = 0.5).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')
ridge.coef_

MAE: 8850581.288715674
MSE: 219126248259251.78
RMSE: 14802913.505768105
MAPE: 0.7287084712002042
R^2: 0.6257232430102068


array([ 2.21975480e+04, -8.06934026e+07, -7.38355669e+07,  1.20515227e+05,
        7.87320755e+06, -3.36680437e+04, -3.36680437e+04, -3.36680437e+04,
       -2.24321394e+06,  2.24321394e+06,  1.62813218e+05, -1.62813218e+05,
        2.35150075e+06, -1.04341390e+06, -1.30808685e+06,  5.39626133e+04,
       -5.39626133e+04])

In [38]:
#L2
lasso = Lasso(alpha = 0.5).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')
lasso.coef_

MAE: 8850581.288715674
MSE: 219126248259251.78
RMSE: 14802913.505768105
MAPE: 0.7287084712002042
R^2: 0.6257232430102068


array([ 2.21740896e+04, -8.14873815e+07, -7.61009161e+07,  1.36801235e+05,
        7.84199240e+06, -9.05495077e+04, -4.03023933e-10, -1.57170553e-10,
       -4.84594282e+06,  1.52310410e-09,  3.18682000e+05, -1.53028440e-10,
        3.59745265e+06,  2.36152774e+05, -3.18686966e+04,  8.91145131e+04,
       -0.00000000e+00])

In [40]:
parameters = {'alpha' : np.arange(0, 1, 0.1)}
#Поиск гиперпараметра с помощью GridSearchCV
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.1}

In [45]:
ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.1}